In [6]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [7]:
import os
path= "/content/drive/My Drive/engineering/college/term 10/pattern recognition/projects/face_recognition" #Alaa
os.chdir(path)
!ls 

face_recognition.ipynb	orl_faces  Untitled0.ipynb


In [0]:

from PIL import Image
import numpy as np
import glob
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


In [0]:



def read_dataset(path):
  data= np.zeros([400,10305])
  i=0
  iamges_list = []
  for filename in glob.glob(path+"/*/*.pgm"): 

    im=Image.open(filename)
    iamges_list.append(im)
    l=regex.split(r'/', filename)
    l2= regex.split(r's', l[1])
    arr = np.array(im)
    data[i,0:10304]=arr.reshape(1,10304)
    data[i, -1]=l2[1]
    i+=1
  return data, iamges_list


    

In [0]:
def data_splitting():
  test = np.zeros([200,10305])
  train = np.zeros([200,10305])
  for i in range(data.shape[0]):
    if i%2 == 0:
      test[int(i/2)]=data[i,:]
    else:
      train[int((i-1)/2)]=data[i]
  return train, test


In [0]:
def visualize(images,i):
  plt.imshow(images[i])
  

In [0]:
data, images= read_dataset("orl_faces") 
train, test = data_splitting()
visualize(images, 32)

In [0]:
def pca():
  pca = PCA()
  pca.fit(data[:,0:-2])
  print(pca.explained_variance_ratio_)  
  new_dimensions= pca.fit_transform(x)
  new_dimensions.shape
  return  new_dimensions

pca ()


